# 2 - Limpeza de dados

O dataset `Air Quality NYC` possui os índices de gases emitidos na cidade de NY e suas principais fontes. Também possui dados hospitalares causados por esses poluentes, tráfegos de caminhões e carros. Ha duas separações de dados para os indicadores, aqueles medidos sazonalemente (verão e inverno), e aqueles medidos anualmente:

**Sazonais (Verão/Inverno):**

| Indicator ID | Name |
| :--- | :--- |
| 375 | Nitrogen dioxide (NO2) |
| 386 | Ozonone (O3) |
| 365 | Fine particles (PM 2.5) |

**Anuais:**

| Indicator ID | Name |
| :--- | :--- |
| 647 | Outdoor Air Toxics - Formaldehyde |
| 646 | Outdoor Air Toxics - Benzene |
| 651 | Cardiovascular hospitalizations due to PM2.5 (age 40+) |
| 652 | Cardiac and respiratory deaths due to Ozone |
| 650 | Respiratory hospitalizations due to PM2.5 (age 20+) |
| 659 | Asthma emergency departments visits due to Ozone |
| 661 | Asthma hospitalizations due to Ozone |
| 657 | Asthma emergency department visits due to PM2.5 |
| 639 | Deaths due to PM2.5 |
| 653 | Asthma emergency departments visits due to Ozone |
| 655 | Asthma hospitalizations due to Ozone |
| 648 | Asthma emergency department visits due to PM2.5 |
| 644 | Annual vehicle miles traveled (cars) |
| 645 | Annual vehicle miles traveled (trucks) |
| 643 | Annual vehicle miles traveled |
| 642 | Boiler Emissions- Total NOx Emissions |
| 641 | Boiler Emissions- Total PM2.5 Emissions |
| 640 | Boiler Emissions- Total SO2 Emissions |

### Estratégias para limpeza de dados

**Missing values**: Não é todo indicador que possui dado de todos os anos (2005-2023). Dados sazonais possuem de 2008-2023, e indicadores anuais possuem de 2005-2019. Infelizmente não temos o que fazer, alguns dados são medidas de um período maior (2012-2014), apenas distribuimos a mesma média para os anos 2012, 2013 e 2014.

**Outliers**: Não identificamos nenhum outlier no dataset;

**Inconsistência**: Não identificamos inconsistências no dataset;

**Padronização**: Os indicadores sazonais terão a coluna inicial `Time Period` quebrada em duas: `Season` e `Year`. Como o inverno começa em um ano e termina em outro, consideraremos o ano em que ele termina para manter a padronização em certos dados. Os indicadores anuais possuem dois formatos: YYYY e YYYY-YYYY. Os anos que estão em um intervalo, separamos e copiamos a coluna `Data Value` para todos o intervalo.

Os dados foram coletados em diferentes regiões da cidade, mas para o nosso uso, agruparemos apenas na cidade. A coluna `Geo Type Name` diz qual a granularidade da informação. Alguns são agregados de outras regiões, `Citywide` por exemplo, representa a média para a cidade como um todo, já a média dos 5 `Borough` é o equivalente à `Citywide`, e assim por diante. Por esse motivo, utilizaremos apenas os registros de `Citywide`.

In [10]:
import pandas as pd
import os

input_path = "./air_quality.parquet"
out_folder = "out"

os.makedirs(out_folder, exist_ok=True)
df = pd.read_parquet(input_path, engine='fastparquet')
df.rename(columns={"Indicator ID": "IndicatorID", "Measure Info": "MeasureInfo", "Data Value": "DataValue", "Geo Type Name": "GeoTypeName"}, inplace=True)

## Preparando dataset dos indicadores de média

In [11]:
# Filtrar os indicadores de média
df_season = df[df["IndicatorID"].isin([375, 386, 365])]

Agora precisamos separar todos os dados por estação do ano e ano. Ou seja, Time Period irá virar as colunas Season e Year

Casos diferentes de Time Period e como tratamos: \
    - `Summer YYYY` : Apenas separamos em duas colunas. \
    - `Winter YYYY-YY`: Como o inverno começa em um ano e termina em outro, consideramos o ano em que ele começa, então mantemos o primeiro ano.


In [12]:
mask_season = (
    df_season['Time Period'].str.startswith('Winter') |
    df_season['Time Period'].str.startswith('Summer')
)


df_season = df_season[mask_season].copy()
# Summer YYYY é simples, apenas separar em duas colunas
df_season[['Season', 'Year']] = df_season['Time Period'].str.split(' ', expand=True)

# Winter YYYY-YY, manter apenas o segundo ano
df_season['Year'] = df_season['Year'].str.split('-', expand=True)[0]
df_season['Year'] = df_season['Year'].astype(int)
mask_winter = (df_season['Season'] == 'Winter')
df_season.loc[mask_winter, 'Year'] = df_season.loc[mask_winter, 'Year'] + 1
df_season.drop(columns=['Time Period'], inplace=True)


df_season = df_season.groupby(["IndicatorID", "Name", "Measure", "MeasureInfo", "GeoTypeName", "Season", "Year"], as_index=False)["DataValue"].mean()
df_season = df_season[df_season["GeoTypeName"] == "Citywide"]
df_season.drop(columns=["GeoTypeName"], inplace=True)
df_season.to_parquet(f"{out_folder}/seasonal_air_quality.parquet", index=False)
df_season

,IndicatorID,Name,Measure,MeasureInfo,Season,Year,DataValue
60,365,Fine particles (PM 2.5),Mean,mcg/m3,Summer,2009,10.700000
61,365,Fine particles (PM 2.5),Mean,mcg/m3,Summer,2010,11.830000
62,365,Fine particles (PM 2.5),Mean,mcg/m3,Summer,2011,11.460000
63,365,Fine particles (PM 2.5),Mean,mcg/m3,Summer,2012,10.310000
64,365,Fine particles (PM 2.5),Mean,mcg/m3,Summer,2013,10.150000
...,...,...,...,...,...,...,...
340,386,Ozone (O3),Mean,ppb,Summer,2019,30.550000
341,386,Ozone (O3),Mean,ppb,Summer,2020,29.840000
342,386,Ozone (O3),Mean,ppb,Summer,2021,30.176713
343,386,Ozone (O3),Mean,ppb,Summer,2022,34.078943


## Preparando dataset dos indicadores anuais

Os indicadores que possuem dados anuais possuem dois formatos:

In [13]:
# Pegando todos os intervalos de anos, para explodi-los
annual_indicators = [
    647, 646, 651, 652, 650, 659, 661, 657, 639, 
    653, 655, 648, 644, 645, 643, 642, 641, 640
]

df_year = df[df["IndicatorID"].isin(annual_indicators)]

# Obtém apenas os anos no formato YYYY-YYYY
mask_year_range = df_year['Time Period'].str.contains(r'^\d{4}-\d{4}$', regex=True)
df_year_range = df_year[mask_year_range].copy()
df_year_range[['Start Year', 'End Year']] = df_year_range['Time Period'].str.split('-', expand=True).astype(int)
df_year_range['Year Count'] = df_year_range['End Year'] - df_year_range['Start Year'] + 1
df_year_range['Year List'] = df_year_range.apply(
    lambda row: list(range(row['Start Year'], row['End Year'] + 1)), axis=1
)

# Explode os intervalos dos anos
df_year_range = df_year_range.explode('Year List').copy()
df_year_range.rename(columns={'Year List': 'Year'}, inplace=True)
df_year_range = df_year_range[["IndicatorID", "Name", "Measure", "MeasureInfo", "GeoTypeName", "Year", "DataValue"]]

# Filtra apenas os anos no formato YYYY
df_year = df_year[~mask_year_range].copy()
df_year = df_year[["IndicatorID", "Name", "Measure", "MeasureInfo", "GeoTypeName","Time Period", "DataValue"]]
df_year.rename(columns={"Time Period": "Year"}, inplace=True)
df_year['Year'] = df_year['Year'].astype(int)

# Juntando os anos individuais com os intervalos explodidos
df_year = pd.concat([df_year_range, df_year], ignore_index=True)
df_year = df_year.groupby(["IndicatorID", "Name", "Measure", "MeasureInfo", "GeoTypeName", "Year"], as_index=False)["DataValue"].mean()
df_year = df_year[df_year["GeoTypeName"] == "Citywide"]
df_year.drop(columns=["GeoTypeName"], inplace=True)
df_year.to_parquet(f"{out_folder}/annual_air_quality.parquet", index=False)
df_year

,IndicatorID,Name,Measure,MeasureInfo,Year,DataValue
14,639,Deaths due to PM2.5,Estimated annual rate (age 30+),"per 100,000 adults",2005,64.70
15,639,Deaths due to PM2.5,Estimated annual rate (age 30+),"per 100,000 adults",2006,64.70
16,639,Deaths due to PM2.5,Estimated annual rate (age 30+),"per 100,000 adults",2007,64.70
17,639,Deaths due to PM2.5,Estimated annual rate (age 30+),"per 100,000 adults",2009,48.20
18,639,Deaths due to PM2.5,Estimated annual rate (age 30+),"per 100,000 adults",2010,48.20
...,...,...,...,...,...,...
475,661,Asthma hospitalizations due to Ozone,Estimated annual rate (age 18+),"per 100,000 adults",2015,3.90
476,661,Asthma hospitalizations due to Ozone,Estimated annual rate (age 18+),"per 100,000 adults",2016,3.90
477,661,Asthma hospitalizations due to Ozone,Estimated annual rate (age 18+),"per 100,000 adults",2017,3.45
478,661,Asthma hospitalizations due to Ozone,Estimated annual rate (age 18+),"per 100,000 adults",2018,3.00
